# XGBoost


## Introdução

<p align = "justify">O XGboost (Extreme Gradient Boosting) é um algoritmo de machine learning que vêm mostrando excelentes resultados, sobretudo em competições do Kagggle. Tal algoritmo é de tipo Ensemble, podendo ser pensado como uma aprimoração do Gradient Boosting. </p>

<p align = "justify"> O Gradient Boosting ajusta os pesos usando o Gradient Descent (em geral olhando para uma loss function específica: MSE para algoritmos de regressão e uma função logarítmica para algoritmos de classificação), otimizando iterativamente a diferença entre o valor previsto e o valor real. Porém, o Gradient Boosting é sequencial, não podendo ser usado em computação paralela e distribuída. Isso motiva a construção do XGBoost que será apresentada ao longo desse material. </p>

<p align = "justify"> Em essência, o XGBoost é um algoritmo que usa a expansão de Taylor de ordem 2 para tratar a função objetivo, tomando árvores de decisão como seus preditores. Além disso, a função objetivo do XGBoost conta com um parâmetro extra de regularização em sua construção, o qual aumenta a precisão, penaliza modelos muito complexos em detrimento de modelos mais simples e evita o temido sobreajuste (ideia que lembra muito a Ridge Regression).</p>

<p align = "justify"> O que faz do XGBoost tão importante não é apenas os seus excelentes resultados mas, tão importante quanto, é a escalabilidade do modelo, uma vez que permite computação paralela e distribuida, permitindo inclusive a construção de modelos out-of-core: o sistema é executado mais de dez vezes mais rápido do que as soluções populares existentes em uma única máquina e pode ser dimensionado para bilhões de exemplos em configurações distribuídas ou com limitação de memória.</p>






## Fundamentação Teórica

### Função objetivo regularizada


<p align = "justify"> Seja $\mathcal{D}=\{ (x_i,y_i)\}_{i=1}^{n}⊂\mathbb{R}^m \times \mathbb{R}$ uma amostra rotulada. Um modelo emsemble de árvore de decisão usa um número $K$ de funções aditivas $f_k$ para predizer o parâmetro $y_i$. Essa predição é denotada por $p_i$ e </p>

$$p_i = \sum_{k=1}^{K} f_k(x_i),\qquad  f_k ∈ \mathcal{F},$$

em que $\mathcal{F} = \{f \mid f(x) = w_{q(x)},\, q:\mathbb{R}^m\to
\{ 1, \cdots, T\} \text{ e } w \in \mathbb{R}^T\}.$

<p align = "justify"> Cada $f_k$ corresponde a uma estrutura de árvore independente $q$ e pesos de folha $w$, com $T$ sendo o número de folhas da árvore.<b> Ao contrário das árvores de decisão, cada árvore de regressão contém uma pontuação contínua em cada folha: </b> os $w_i$'s são usados para representar a pontuação na $i$-ésima folha. Para cada $x_i$, usaremos as regras de decisão nas árvores (dadas por $q$) para classificá-lo nas folhas e calcular a previsão final somando a pontuação nas folhas correspondentes (dadas por $w$). Para aprender o conjunto de funções usadas no modelo, se minimiza o objetivo regularizado: </p>

$$\mathcal{L} =  \sum_{i} l(y_i,p_i) +\sum_{k}\Omega(f_k)$$
onde $$\Omega(f) = \Omega(w_q(x) ) = \gamma T + \frac{1}{2}\lambda \|w\|^2.$$ 

<p align = "justify">
Aqui $l$ é uma "loss function" (convexa e diferenciável) que mede a diferença entre a previsão $p_i$ e o alvo $y_i$. O segundo termo $\Omega$ penaliza a complexidade do modelo (ou seja, as funções da árvore de regressão). Esse termo de regularização ajuda a suavizar os pesos finais aprendidos para evitar sobreajuste (mesma ideia utilizada na construção do ridge regression - para mais detalhes sobre esse algoritmo, veja o curso na udemy: machine learning, do Prof. Edson Cilos). Intuitivamente, o objetivo regularizado tenderá a selecionar um modelo empregando funções simples e preditivas. Quando $\Omega = 0$, o problema acima coincide com o objetivo do GBoost.


### O algoritmo XGBoost


<p align = "justify"> Seja $p_i^{(t)}$ a previsão da $i$-ésima instância na $t$-ésima iteração. Por construção, fazemos $p_i^{(t)}(x_i) = p_i^{(t - 1)}(x_i) + f_{t}(x_i)$, portanto, acrescentamos na iteração $t$, para cada $x_i$, 
o termo $f_{t}(x_i)$. Na prática queremos encontrar  $f_t$ afim de minimizar o problema

$$\mathcal{L}^{(t)} =  \sum_{i= 1}^{n} l(y_i,p_i^{(t-1)}+f_t(x_i)) +\Omega(f_t)$$


<p align = "justify">O que é equivalente a encontrar o peso $w_{q(x)} = f(x)$ para cada uma das folhas, isto é, encontrar cada um dos $w_i$'s que é o peso associado à $i$-ésima folha, com $i \in \{1, \cdots, T \}.$




Usando a expansão de Taylor de ordem 2, sabemos que 

$$f(a+h)≈ f(a)+f'(a)h+\frac{1}{2}f''(a)h^2$$ para $f\in C^2$. Se $l$ for suficientemente suave, fazemos $f=l$, $a = p_i^{(t)}$ e $h=f_t(x_i)$, obtendo assim

\begin{align}
\mathcal{L}^{(t)} &≈  \sum_{i= 1}^{n} \bigl( l(y_i,p_i^{(t-1)})+g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2\bigr) +\Omega(f_t) \\ &= {C} + \sum_{i= 1}^{n} \bigl[ g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2\bigr) +\Omega(f_t)
\end{align}
em que $C = \sum_{i}^{n} l(y_i,p_i^{(t-1)}) $ é uma constante uma vez que essa expressão já é conhecido na iteração $t - 1$. Além disso: $$g_i = \frac{\partial l(y_i, p_i^{(t-1)})}{\partial p_i^{(t-1)}} \qquad \text{ e } \qquad h_i = \frac{\partial^2 l(y_i, p_i^{(t-1)})}{\partial p_i^{(t-1)}\,^2 }.$$


<p align = "justify"> Removendo o termo constante $C$, nossa nova função objetivo se torna no passo $t$ igual a 

$$\mathcal{L_1}(t) = \Omega(f_t)+ \sum_{i= 1}^{n} \bigl(g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2\bigr).$$


<p align = "justify">Dado $j\in\{1,\dots, T\}$, defina $I_j  = \{i\mid q(x_i)=j\}$. Expandindo $\Omega$ e retomando definição de $\mathcal{F}$, podemos escrever
\begin{align}
\mathcal{L}_1^{(t)} &=\gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T} w_j^2+\sum_{i= 1}^{n} g_if_t(x_i)+\frac{1}{2}h_i [f_t(x_i)]^2 \\& = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T} w_j^2+ \sum_{j= 1}^{T}\sum_{i \in I_j} g_iw_j+\frac{1}{2}\sum_{j= 1}^{T}\sum_{i \in I_j} h_i[w_j]^2\\& = \gamma T + \sum_{j=1}^{T} \biggl[ \sum_{i \in I_j} g_iw_j+ \frac{1}{2}w_j^2\biggl( \lambda + \sum_{i \in I_j} h_i \biggr)\biggr]
\end{align}

<p align = "justify"> <b> Uma vez fixada a estrutura $\boldsymbol{q(x)}$</b>, podemos calcular para cada $j$ o peso ótimo $w_j^*$. Fixe $j \in \{1,\dots,T \}$. A ideia é relativamente simples: para minimizarmos a soma, basta minimizarmos cada um de seus fatores. Portanto, queremos minimizar
\begin{equation*}
R(w_j) = \sum_{i \in I_j} g_iw_j+ \frac{1}{2}w_j^2\biggl( \lambda + \sum_{i \in I_j} h_i \biggr).
\end{equation*}
<p align = "justify"> Sendo $l$ uma função convexa, segue que $h_i\geq 0$ para qualquer $i\in I_j$. Assim, o mínimo da parábola $R(w_j)$ é dado no ponto $w_j^*$ tal que $R'(w_j^*)=0$. Logo
\begin{equation*}
0 = R'(w_j^*) = \sum_{i \in I_j} g_i+w_j^*\biggl( \lambda + \sum_{i \in I_j} h_i \biggr),
\end{equation*}
ou seja, o peso $w_j^*$ ótimo é dado por
$$
w_j^* = -\frac{\sum_{i \in I_j} g_i}{\lambda + \sum_{i \in I_j} h_i}
$$
<p align = "justify">Por conseguinte, o valor ótimo correspondente é </p>

$$
\mathcal{L}_1^{(t)}(q) = \gamma T-\frac{1}{2}\sum_{j=1}^{T}\frac{\bigl(\sum_{i \in I_j} g_i\bigr)^2}{\lambda + \sum_{i \in I_j} h_i},
$$


<p align = "justify">para cada estrutura de árvore fixa $q$. Adicionalmente, para cada estrutura de árvore $q$ você pode usar a expressão encontrada para $\mathcal{L}_1^{(t)}(q)$ como uma medida da "perda" da árvore $q$, como se medisse o grau de impureza.

<p align = "justify"> Todavia, de modo geral, não é possível enumerar todas as estruturas de árvores $q$ à fim de encontrar aquela estrutura que forneça a menor impureza </font>. Em vez disso, usa-se um algoritmo ambicioso que começa a partir de uma única folha e adiciona iterativamente ramos à árvore. Se $I_L$ e $I_R$ são os conjuntos de instâncias dos nós esquerdo e direito após a divisão, então deixando $I = I_L \cup I_R$ a redução da perda após a divisão é dada por</p>

\begin{align}
\mathcal{L}_{split} &= \gamma -\frac{1}{2}\frac{\bigl(\sum_{i \in I} g_i\bigr)^2}{\lambda + \sum_{i \in I} h_i} - \biggl[\gamma-\frac{1}{2}\frac{\bigl(\sum_{i \in I_L} g_i\bigr)^2}{\lambda + \sum_{i \in I_L} h_i} + \gamma -\frac{1}{2}\frac{\bigl(\sum_{i \in I_R} g_i\bigr)^2}{\lambda + \sum_{i \in I_R} h_i} \biggr]\\&=\frac{1}{2}\biggl[ \frac{\bigl(\sum_{i \in I_L} g_i\bigr)^2}{\lambda + \sum_{i \in I_L} h_i} +\frac{\bigl(\sum_{i \in I_R} g_i\bigr)^2}{\lambda + \sum_{i \in I_R} h_i} -\frac{\bigl(\sum_{i \in I} g_i\bigr)^2}{\lambda + \sum_{i \in I} h_i}\biggr]-\gamma.
\end{align}


### Pseudocódigo do XGBoost - versão em série


<p align = "justify"> Input: conjunto de treinamento $\{(x_{i},y_{i})\}_{i=1}^{n}$, uma função de perda diferenciável $l(y, F(x))$ de classe $C^2$, um número $M$ de modelos de predição mais simples (árvores de decisão, por exemplo) e uma taxa de aprendizagem $\alpha$.

Algoritmo:

1. Inicializar o modelo com um valor constante:

$${p^{(0)}={\underset {\theta }{\arg \min }}\sum _{i=1}^{n}l(y_{i},\theta ).}$$

2. Para $m = 1$ até $M$:
    1. Computar os "gradientes" e as "hessianas":
\begin{align}
{\hat {g}}_{m}(x_{i})&=\left[{\frac {\partial l(y_{i},f(x_{i}))}{\partial f(x_{i})}}\right]\biggl|_{p^{(m-1)}}\\
{\hat {h}}_{m}(x_{i})&=\left[{\frac {\partial ^{2}l(y_{i},f(x_{i}))}{\partial f(x_{i})^{2}}}\right]\biggl|_{p^{(m-1)}}.
\end{align}
    2. Ajustar a base de aprendizado usando o conjunto de treinamento
$\left\{x_{i},-\frac{\hat g_m (x_{i})}{ \hat {h}_{m}(x_{i})}\right\}_{i=1}^{n}$ resolvendo o problema de otimização:
$$
{\displaystyle {\hat {\phi }}_{m}={\underset {\phi \in \mathbf {\Phi } }{\arg \min }}\sum _{i=1}^{n}{\frac {1}{2}}{\hat {h}}_{m}(x_{i})\left[-{\frac {{\hat {g}}_{m}(x_{i})}{{\hat {h}}_{m}(x_{i})}}-\phi (x_{i})\right]^{2}.}
$$
$$
{\displaystyle f_{m}(x)=\alpha {\hat {\phi }}_{m}(x).}
$$
 
 3. Atualizar o modelo:
$$
{p^{(m)}={p}^{(m-1)}+{{f}}_{m}(x).}
$$
3. Output: $p^{(M)}.$


<p align = "justify"> <b>Observação: </b> Existem implementações do XGoost para computação paralela e distribuída, as quais estão fora do escopo do trabalho.

## Prós e contras.

Prós:
*   Alta velocidade quando comparado com outros algorítmos.
*   Grande desempenho para dados tabulares/estruturados.
*   O termo de regularização do algorítmo penaliza a construção de árvores complexas, evitando o sobreajuste.
*   Pode lidar com valores ausentes.
*   Não há necessidade de dimensionar/normalizar dados.
*   Admite implementação escalável, podendo ser usar computação paralela e distribuída.

Contras:
*   Não explora todas as estruturas de árvore possíveis.
*   Pode deixar a desejar se os dados não forem estruturados (ex. imagens).
*   Se os parâmetros da função objetivo não forem ajustados corretamente, pode facilmente sobreajustar os dados.


## Comparando o XGboost com outros algorítmos

In [19]:
import pandas as pd
import numpy as np
import os

In [20]:
seed = 42
np.random.seed(seed)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

### Regressão - Dataset California Housing

O objetivo aqui é predizer valores de imóveis.




In [22]:
url_ch1 = 'https://raw.githubusercontent.com/V4k0s/XGBoost/master/Datasets/california_housing_train.csv'

housing = pd.read_csv(url_ch1)

In [23]:
X = housing.drop("median_house_value", axis=1)
Y = housing['median_house_value']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                        test_size=0.2, 
                                        random_state=seed)

In [24]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(max_features=8, n_estimators=30, random_state=seed)
forest_scores = cross_val_score(forest_reg, X_train, Y_train,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [53411.71987219 50601.07815024 47668.02059031 53267.30741694
 47343.05818952 48416.20612185 53355.49522704 43516.46869185
 51094.49428672 50324.30086461]
Mean: 49899.814941125354
Standard deviation: 3036.124845297608


In [25]:
from xgboost import XGBRegressor
xg_reg = XGBRegressor(learning_rate=0.08, 
                                 n_estimators=1300,
                                 max_depth=6,
                                 random_state=seed,
                                 gpu_id='0',
                                 booster="gbtree",
                                 warm_start = True,
                                 objective='reg:squarederror'
                                 ) 

xg_scores = cross_val_score(xg_reg, X_train, Y_train,
                             scoring="neg_mean_squared_error", cv=10)
xg_rmse_scores = np.sqrt(-xg_scores)
display_scores(xg_rmse_scores)

Scores: [48083.02355567 45591.34735867 44351.16884563 48411.29139488
 44264.62137323 43599.57509329 48203.12307105 42048.7401153
 49217.61988536 47014.6680959 ]
Mean: 46078.51787889657
Standard deviation: 2315.1612823618702


In [26]:
xg_reg.fit(X_train, Y_train)
y_pred = xg_reg.predict(X_test)

np.sqrt(MSE(Y_test, y_pred))

46653.07933960706

###Classificação - Dataset College

O objetivo aqui é predizer se uma determinada escola é particular ou pública.

In [27]:
url_cll = 'https://raw.githubusercontent.com/V4k0s/XGBoost/master/Datasets/college.csv'
        
college = pd.read_csv(url_cll)

X = college.drop('private', axis=1)
Y = college.private

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                        test_size=0.2, 
                                        random_state=seed)

In [28]:
from sklearn.tree import DecisionTreeClassifier
trclf = DecisionTreeClassifier(random_state=seed)

trclf_scores = cross_val_score(trclf, X_train, Y_train, cv=10, scoring='accuracy')
print(np.mean(trclf_scores))

0.8922171018945212


In [29]:
from xgboost import XGBClassifier
xgclf = XGBClassifier(random_state=seed)

xgclf_scores = cross_val_score(xgclf, X_train, Y_train, cv=10, scoring='accuracy')
print(np.mean(xgclf_scores))

0.9389144905273937


In [30]:
xgclf.fit(X_train, Y_train)
Y_pred = xgclf.predict(X_test)

print(accuracy_score(Y_test,Y_pred))

0.9294871794871795


###Classificação - Dataset Titanic

O objetivo aqui é predizer se determinada pessoa vai sobreviver ao desastre ocorrido com o navio Titanic.

In [31]:
url_tc1 = 'https://raw.githubusercontent.com/V4k0s/XGBoost/master/Datasets/titanic_train.csv'


titanic = pd.read_csv(url_tc1)

In [32]:
## Tratamento dos dados ##
titanic.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
titanic['Age'].fillna(titanic['Age'].mean(),inplace=True)
titanic['Fare'] = titanic['Fare'].replace(0, titanic['Fare'].mean())
titanic['Fare'].replace('nan',np.nan,inplace=True)
titanic['Fare'].fillna(titanic['Fare'].mean(),inplace=True)
titanic['Embarked'].replace('nan',np.nan,inplace=True)
titanic['Embarked'].fillna(titanic['Embarked'].mode()[0],inplace=True)
titanic['Sex']=titanic['Sex'].map({'male':0,'female':1})
titanic['Embarked']=titanic['Embarked'].map({'S':0,'C':1,'Q':2})
titanic['Age']=np.log(titanic['Age'])
titanic['Fare']=np.log(titanic['Fare'])

X = titanic.drop(['Survived'], axis=1)
Y = titanic['Survived']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                        test_size=0.3, 
                                        random_state=seed)

In [33]:
from sklearn.tree import DecisionTreeClassifier
trclf = DecisionTreeClassifier(random_state=seed)

trclf_scores = cross_val_score(trclf, X_train, Y_train, cv=10, scoring='accuracy')
print(np.mean(trclf_scores))

0.7626216077828981


In [34]:
from xgboost import XGBClassifier

xgclf = XGBClassifier(random_state=seed)
xgclf_scores = cross_val_score(xgclf, X_train, Y_train, cv=10, scoring='accuracy')

print(np.mean(xgclf_scores))

0.8282130056323604


In [35]:
grid_param = {  
    'n_estimators': [10, 50, 75, 115],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
    'early_stopping_rounds': [2, 3, 4, 5],
    'warm_start': [True]}
    
g_search = GridSearchCV(estimator = xgclf, param_grid = grid_param, 
                     cv = 3, n_jobs = -1, verbose = 2)

g_search.fit(X_train, Y_train)
print(g_search.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
{'early_stopping_rounds': 2, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 75, 'warm_start': True}


In [36]:
g_search.score(X_test,Y_test)

0.8171641791044776